In [1]:
# =========================
# INSTALLS (if needed)
# =========================
# !pip install -U sentence-transformers scikit-learn torch transformers tqdm

# =========================
# IMPORTS
# =========================
import numpy as np
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import os, random, torch, numpy as np

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    ##torch.backends.cudnn.deterministic = False
    ##torch.backends.cudnn.benchmark = True
    torch.use_deterministic_algorithms(True, warn_only=False)
    print(f"[INFO] All seeds set to {seed}")

seed_everything(42)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

def free_gpu():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
##set_seed(42)
# =========================
# CONFIG
# =========================



MODELS_NAMES= ["aubmindlab/bert-base-arabert","UBC-NLP/MARBERTv2",
              "aubmindlab/araelectra-base-generator",
               "3ebdola/Dialectal-Arabic-XLM-R-Base", 
               "aubmindlab/araelectra-base-discriminator",
               "bashar-talafha/multi-dialect-bert-base-arabic",
              "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
              "sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking",
              "sentence-transformers/distiluse-base-multilingual-cased",
               "sentence-transformers/clip-ViT-B-32-multilingual-v1",
               "sentence-transformers/distiluse-base-multilingual-cased-v1",
               "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
              ]


NUM_EPOCHS = 3
BATCH_SIZE = 8
LR = 2e-5
N_SPLITS = 5
MAX_LEN = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================
# LOAD YOUR DATA
# =========================
# Example:
df = pd.read_csv("/kaggle/input/ekafnewsforkhawla/shuffled_cleaned_text_file for khawla.txt",sep='\t',encoding="utf-8")
df = df[["preprocess1", "label", "dataset"]]
# df must contain: text | label

# Dummy placeholder (REMOVE THIS)
# df = pd.DataFrame({
#     "text": ["example text one", "example text two"],
#     "label": [0, 1]
# })

texts = df["preprocess1"].astype(str).values
labels = df["label"].astype(int).values
num_labels = len(np.unique(labels))

# =========================
# DATASET CLASS
# =========================
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }
for MODEL_NAME in MODELS_NAMES:
    # =========================
    # TOKENIZER
    # =========================
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # =========================
    # CROSS VALIDATION
    # =========================
    strat = df['label']

    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=False, random_state=None)


    all_preds = []
    all_labels = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(texts, strat), 1):
        free_gpu()
        print(f"\n{'='*30}")
        print(f"FOLD {fold}")
        print(f"{'='*30}")
        
        # Datasets
        train_ds = TextDataset(texts[train_idx], labels[train_idx], tokenizer)
        val_ds   = TextDataset(texts[val_idx], labels[val_idx], tokenizer)

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False)
        val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

        # Model
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=num_labels
        ).to(DEVICE)

        optimizer = AdamW(model.parameters(), lr=LR)

        # =========================
        # TRAINING
        # =========================
        model.train()
        for epoch in range(NUM_EPOCHS):
            loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
            for batch in loop:
                optimizer.zero_grad()

                outputs = model(
                    input_ids=batch["input_ids"].to(DEVICE),
                    attention_mask=batch["attention_mask"].to(DEVICE),
                    labels=batch["labels"].to(DEVICE)
                )

                loss = outputs.loss
                loss.backward()
                optimizer.step()

                loop.set_postfix(loss=loss.item())

        # =========================
        # VALIDATION
        # =========================
        model.eval()
        fold_preds = []
        fold_labels = []

        with torch.no_grad():
            for batch in val_loader:
                outputs = model(
                    input_ids=batch["input_ids"].to(DEVICE),
                    attention_mask=batch["attention_mask"].to(DEVICE)
                )
                preds = torch.argmax(outputs.logits, dim=1)

                fold_preds.extend(preds.cpu().numpy())
                fold_labels.extend(batch["labels"].numpy())

        # Store global results
        all_preds.extend(fold_preds)
        all_labels.extend(fold_labels)

        # =========================
        # REPORTS PER FOLD
        # =========================
        print("\nClassification Report (Fold {})".format(fold))
        print(classification_report(fold_labels, fold_preds, digits=4))

        print("Confusion Matrix (Fold {})".format(fold))
        print(confusion_matrix(fold_labels, fold_preds))

    # =========================
    # FINAL GLOBAL RESULTS
    # =========================
    print(f"\n{'✅'*50}")
    print("OVERALL RESULTS (ALL FOLDS)")
    print(MODEL_NAME)
    print(f"{'✅'*50}")

    print("\nFinal Classification Report")
    print(classification_report(all_labels, all_preds, digits=4))

    print("Final Confusion Matrix")
    print(confusion_matrix(all_labels, all_preds))
    print(f"\n{'✅'*50}")


[INFO] All seeds set to 42


tokenizer_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...


Classification Report (Fold 3)
              precision    recall  f1-score   support

           0     0.8160    0.7473    0.7801       273
           1     0.9113    0.9391    0.9250       755

    accuracy                         0.8881      1028
   macro avg     0.8637    0.8432    0.8525      1028
weighted avg     0.8860    0.8881    0.8865      1028

Confusion Matrix (Fold 3)
[[204  69]
 [ 46 709]]

FOLD 4


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 3: 100%|██████████| 514/514 [00:54<00:00,  9.37it/s, loss=0.0199] 



Classification Report (Fold 4)
              precision    recall  f1-score   support

           0     0.7331    0.7978    0.7641       272
           1     0.9248    0.8954    0.9098       755

    accuracy                         0.8695      1027
   macro avg     0.8289    0.8466    0.8370      1027
weighted avg     0.8740    0.8695    0.8712      1027

Confusion Matrix (Fold 4)
[[217  55]
 [ 79 676]]

FOLD 5


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 3: 100%|██████████| 514/514 [00:54<00:00,  9.36it/s, loss=0.0312] 



Classification Report (Fold 5)
              precision    recall  f1-score   support

           0     0.8008    0.7096    0.7524       272
           1     0.8995    0.9364    0.9176       755

    accuracy                         0.8763      1027
   macro avg     0.8502    0.8230    0.8350      1027
weighted avg     0.8734    0.8763    0.8738      1027

Confusion Matrix (Fold 5)
[[193  79]
 [ 48 707]]

✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅
OVERALL RESULTS (ALL FOLDS)
sentence-transformers/paraphrase-multilingual-mpnet-base-v2
✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅

Final Classification Report
              precision    recall  f1-score   support

           0     0.8113    0.7296    0.7683      1361
           1     0.9060    0.9388    0.9221      3777

    accuracy                         0.8834      5138
   macro avg     0.8586    0.8342    0.8452      5138
weighted avg     0.8809    0.8834    0.8814      5138

Final Confusion Matrix
[[ 993  368]
 [ 231 35